<a href="https://colab.research.google.com/github/SergSis/audio2text_bot/blob/main/audio2text_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot nest_asyncio transformers librosa --quiet

In [19]:
import nest_asyncio
nest_asyncio.apply()

import os
import logging
import sys
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa
from transformers import pipeline

In [25]:
import logging

# Отключаем INFO-логи для httpx
logging.getLogger("httpx").setLevel(logging.WARNING)

# Также можно отключить логи библиотеки httpcore, которая используется внутри httpx
logging.getLogger("httpcore").setLevel(logging.WARNING)

if not logger.handlers:
    # Если обработчиков нет, добавляем новый обработчик
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s — %(name)s — %(levelname)s — %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
else:
    # Если обработчики уже есть, выводим предупреждение
    print("Обработчик уже добавлен.")

In [ ]:
import os
import logging
import librosa
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class TranscriberBot:
    def __init__(self, token: str, model_name: str = "waveletdeboshir/whisper-small-ru-pruned-ft"):
        """
        Инициализация бота: загружаем модель и настраиваем Telegram.
        """
        self.token = token
        logger.info("Загружаем модель ASR для русского")
        self.processor = WhisperProcessor.from_pretrained(model_name)
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name)
        logger.info("Модель загружена")

        self.app = ApplicationBuilder().token(self.token).build()
        self._register_handlers()

    def _register_handlers(self):
        """
        Регистрируем обработчики /start и аудио.
        """
        self.app.add_handler(CommandHandler("start", self.start))
        self.app.add_handler(MessageHandler(filters.VOICE | filters.AUDIO, self.handle_audio))

    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """
        /start: приветствие и инструкция.
        """
        user = update.effective_user
        logger.info(f"/start от {user.id}")
        await update.message.reply_text("Привет! 🤖 Отправь голосовое или аудиофайл — я верну его текст на русском.")

    async def handle_audio(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """
        Обработка аудио: скачиваем, транскрибируем, отвечаем, удаляем файл.
        """
        user = update.effective_user
        msg = update.message

        if msg.voice:
            file_id, ext = msg.voice.file_id, 'ogg'
        elif msg.audio:
            file_id = msg.audio.file_id
            ext = os.path.splitext(msg.audio.file_name or 'audio.mp3')[1].lstrip('.') or 'mp3'
        else:
            return

        logger.info(f"Получено аудио от {user.id}: {file_id}.{ext}")
        tg_file = await context.bot.get_file(file_id)
        os.makedirs("audios", exist_ok=True)
        local_path = f"audios/{file_id}.{ext}"
        await tg_file.download_to_drive(local_path)
        logger.info(f"Сохранён: {local_path}")

        status_message = await update.message.reply_text("✔️ Аудиофайл получен — распознаю...")

        try:
            audio_arr, sr = librosa.load(local_path, sr=16000)
            inputs = self.processor(audio_arr, sampling_rate=sr, return_tensors="pt")
            predicted_ids = self.model.generate(**inputs)
            text = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
            text = text.lstrip('-').strip()
            logger.info(f"Результат транскрипции: {text[:100]}")

            await status_message.edit_text("✅ Распознавание завершено.")
            await update.message.reply_text(text)

        except Exception as e:
            logger.error(f"Ошибка транскрипции: {e}", exc_info=True)
            await status_message.edit_text("⚠️ Не удалось распознать аудио.")
            await update.message.reply_text("⚠️ Не удалось распознать аудио.")

        finally:
            # Удаляем локальный файл, независимо от успеха транскрипции
            try:
                os.remove(local_path)
                logger.info(f"Файл удалён: {local_path}")
            except Exception as err:
                logger.warning(f"Не удалось удалить файл {local_path}: {err}")

    def run(self):
        """
        Запуск Telegram-бота.
        """
        logger.info("Запуск бота...")
        self.app.run_polling()

def load_token(path="token.txt") -> str:
    """
    Читаем токен из файла token.txt.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Не найден файл {path}")
    token = open(path).read().strip()
    if not token:
        raise ValueError("Файл токена пустой")
    return token

if __name__ == "__main__":
    token = load_token()
    bot = TranscriberBot(token)
    bot.run()
